Categorical Features

Transaction

*   ProductCD
*   card1
*   card6
*   addr1
*   addr2
*   P_emaildomain
*   R_emaildomain
*   M1 - M9

Categorical Features

Identity

*   DeviceType
*   DeviceInfo
*   id_01 - id_38


In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import datetime
import gc

In [146]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
#import catboost

Параметры для данного ноутбука:

In [147]:
target = 'isFraud' # Целевой признак
delete_column_list = ['TransactionID','TransactionDT', 'DeviceType','DeviceInfo'] # Список ненужных признаков
is_drop_collinear = False # Удалять ли коррелирующие признаки
coefficient_corr = 0.95 # Коэффициент корреляции, начиная с которого удалять признаки
nan_percent = 0.95 # Степень пустых ячеек, при которой мы относим признак к плохим
is_drop_nan_columns = False # Удалять ли найденные признаки с плохой заполняемостью
bad_fill_category_value = '_NAN_' # Чем заполнять пропуски в категориальных признаках
bad_fill_float_value = 0 # Чем заполнять пропуски в вещественных признаках
bad_fill_numeric_value = 0 # Чем заполнять пропуски в целочисленных признаках
is_need_pca = False #Испольховать ли PCA
max_pca_component = 40 # Количество компонент РСА

#Параметры модели:
CatBoostClassifier_param ={'eval_metric':'AUC'}
"""CatBoostClassifier_param ={
    #'train_dir':'catboost_clf_find_fraud',
    'eval_metric':'AUC',
    'od_pval':0.05,
    'od_wait':20,
    # early_stopping_rounds:20,
    'random_seed':52,    
    'task_type':'CPU',
    # task_type:'GPU',
    # devices:'0',
    'auto_class_weights':'Balanced'#,
    #'use_best_model':True
}"""

is_GridSearchCV = True # Использовать ли GridSearchCV

Загружаем данные:

In [148]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [149]:
def get_data(transaction_csv, identity_csv,is_test = False, delete_column_list=[]):
  df_train_transaction = pd.read_csv(transaction_csv)
  df_train_identity = pd.read_csv(identity_csv)
  df_train = df_train_transaction.merge(df_train_identity, how = 'left',on='TransactionID')
  
  del df_train_identity,df_train_transaction
  gc.collect

  if is_test == False:
    df_y = pd.DataFrame(df_train[target])
    df_train.drop(columns = [target],axis = 1, inplace = True)
  else:
    df_y = pd.DataFrame(df_train['TransactionID'])

  for column in delete_column_list:
    df_train.drop(column, 1,inplace = True)

  return df_train, df_y



Поизучаем данные

In [150]:
df_train, df_y = get_data('gdrive/My Drive/data/gp2/train_transaction.csv', 
                'gdrive/My Drive/data/gp2/train_identity.csv',
                True,[])

Посмотрим на целевую переменную:

In [151]:
df_train[target].value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

# Вывод
Выборка несбалансирована

Проверим корреляцию с целевым признаком

In [152]:
"""correlations = df_train.corr()[target].sort_values()
print(correlations[correlations>0.8], '\n')
print(correlations[correlations<-0.8], '\n')
"""

"correlations = df_train.corr()[target].sort_values()\nprint(correlations[correlations>0.8], '\n')\nprint(correlations[correlations<-0.8], '\n')\n"

Посмотрим на признаки, содержащие пропуски:

In [153]:
def print_count_nan(df):
  row_count = df.shape[0]
  for column in df.columns:
    nan_count = df[column].isna().sum()
    if  nan_count > 0:
      print(column,'  ',nan_count,' (', "{0:.3f}".format(nan_count/row_count), '%)')
      #if row_count == nan_count:
      #  df[column] = 0
    #else:
      #print(column,' нет')

print_count_nan(df_train)

card2    8933  ( 0.015 %)
card3    1565  ( 0.003 %)
card4    1577  ( 0.003 %)
card5    4259  ( 0.007 %)
card6    1571  ( 0.003 %)
addr1    65706  ( 0.111 %)
addr2    65706  ( 0.111 %)
dist1    352271  ( 0.597 %)
dist2    552913  ( 0.936 %)
P_emaildomain    94456  ( 0.160 %)
R_emaildomain    453249  ( 0.768 %)
D1    1269  ( 0.002 %)
D2    280797  ( 0.475 %)
D3    262878  ( 0.445 %)
D4    168922  ( 0.286 %)
D5    309841  ( 0.525 %)
D6    517353  ( 0.876 %)
D7    551623  ( 0.934 %)
D8    515614  ( 0.873 %)
D9    515614  ( 0.873 %)
D10    76022  ( 0.129 %)
D11    279287  ( 0.473 %)
D12    525823  ( 0.890 %)
D13    528588  ( 0.895 %)
D14    528353  ( 0.895 %)
D15    89113  ( 0.151 %)
M1    271100  ( 0.459 %)
M2    271100  ( 0.459 %)
M3    271100  ( 0.459 %)
M4    281444  ( 0.477 %)
M5    350482  ( 0.593 %)
M6    169360  ( 0.287 %)
M7    346265  ( 0.586 %)
M8    346252  ( 0.586 %)
M9    346252  ( 0.586 %)
V1    279287  ( 0.473 %)
V2    279287  ( 0.473 %)
V3    279287  ( 0.473 %)
V4    279287

In [154]:
del df_train, df_y 
gc.collect

<function gc.collect>

Загрузим исходный датасет в трейновую выборку

In [155]:
df_train, df_y = get_data('gdrive/My Drive/data/gp2/train_transaction.csv', 
                'gdrive/My Drive/data/gp2/train_identity.csv',
                False,delete_column_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


Удалим  коррелированные признаки

In [156]:
def remove_collinear_features(x, threshold=0.6):

    
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    
                   
    return x

In [157]:
if is_drop_collinear == True: 
  shape = df_train.shape[1]
  df_trans = remove_collinear_features(df_train, coefficient_corr)
  print('Удалено признаков:')
  print(shape - df_trans.shape[1])

Разделение столбцов по типам и по заполненностью данными:

In [158]:
print(df_train.dtypes)

TransactionAmt    float64
ProductCD          object
card1               int64
card2             float64
card3             float64
                   ...   
id_34              object
id_35              object
id_36              object
id_37              object
id_38              object
Length: 429, dtype: object


In [159]:
from numpy.core.numerictypes import obj2sctype
def create_column_lists(data, max_nan=0.8):
  column_dict={}
  column_dict['categorical_column'] = []
  column_dict['numeric_column'] = []
  column_dict['float_column'] = []
  column_dict['bad_categorical_column'] = []
  column_dict['bad_numeric_column'] = []
  column_dict['bad_float_column'] = []
  for column in data.columns:
    row_count = data[column].shape[0]
    if data[column].isna().sum()/row_count > max_nan:
      if data[column].dtypes == object:
        column_dict['bad_categorical_column'].append(column)
      else:
        if data[column].dtypes == float:
          column_dict['bad_float_column'].append(column)
        else:
          column_dict['bad_numeric_column'].append(column)
    else:
      if data[column].dtypes == object:
        column_dict['categorical_column'].append(column)
      else:
        if data[column].dtypes == float:
          column_dict['float_column'].append(column)
        else:
          column_dict['numeric_column'].append(column)
  return column_dict




In [160]:
column_dict = create_column_lists(df_train, nan_percent)
for key in column_dict:
  print(key+':')
  print(column_dict[key])

categorical_column:
['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
numeric_column:
['card1']
float_column:
['TransactionAmt', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64

Разбираемся с почти пустыми столбцами и заполним значениями остальные

In [161]:
def fill_data(data,column_dict,is_drop_nan = False, is_test = False):
  if is_drop_nan == True:
    data.drop(columns=column_dict['bad_float_column']+
            column_dict['bad_numeric_column']+
            column_dict['bad_categorical_column'],axis = 1,inplace = True)
  else:
    for column in column_dict['bad_categorical_column']:
      data[column].fillna(bad_fill_category_value, inplace = True)
    for column in column_dict['bad_float_column']:
      data[column].fillna(bad_fill_float_value, inplace = True)
    for column in column_dict['bad_numeric_column']:
      data[column].fillna(bad_fill_numeric_value, inplace = True)

  """column_dict['bad_categorical_column']=[]
  column_dict['bad_float_column']=[]
  column_dict['bad_numeric_column']=[]"""

  for column in column_dict['categorical_column']:
    if not(column in column_dict['bad_categorical_column']):
      data[column].fillna(bad_fill_category_value, inplace = True)
  for column in column_dict['float_column']:
    if not(column in column_dict['bad_float_column']):
      data[column].fillna(data[column].mean(), inplace = True)
  for column in column_dict['numeric_column']:
    if not(column in column_dict['bad_numeric_column']):
      data[column].fillna(data[column].mean(), inplace = True)

  if (is_drop_nan == False)and( is_test == False):
    column_dict['categorical_column'] = column_dict['categorical_column']+\
      column_dict['bad_categorical_column']
    column_dict['float_column'] = column_dict['float_column']+\
      column_dict['bad_float_column']
    column_dict['numeric_column'] = column_dict['numeric_column']+\
      column_dict['bad_numeric_column']


  return data, column_dict


In [162]:
df_train,column_dict = fill_data(df_train,column_dict,is_drop_nan_columns, False)

Готовим данные

In [163]:
X_train, X_val, y_train, y_val = train_test_split(df_train, df_y, test_size = 0.3, random_state = 42)


In [164]:
del df_train,df_y
gc.collect

<function gc.collect>

Применение PCA - метода

In [165]:
from sklearn.decomposition import PCA
if is_need_pca == True:
  pca = PCA(max_pca_component)
  pca.fit(X_train[column_dict['float_column']+column_dict['numeric_column']])
  X_train_pca_part1 = pca.transform(X_train[column_dict['float_column']+column_dict['numeric_column']])
  X_val_pca_part1  = pca.transform(X_val[column_dict['float_column']+column_dict['numeric_column']])
  
  X_train_pca_part = pd.DataFrame(X_train_pca_part1,index=X_train.index)
  X_val_pca_part = pd.DataFrame(X_val_pca_part1,index=X_val.index)
  
  X_train_pca_part = X_train_pca_part.join(X_train[column_dict['categorical_column']])
  X_val_pca_part = X_val_pca_part.join(X_val[column_dict['categorical_column']])  

  plt.plot(pca.explained_variance_ratio_.cumsum())
  plt.axvline(x=4)
  plt.grid()


In [166]:
#X_val_pca_part.isna().sum()

CatBoost

In [167]:
!pip install -q catboost shap

In [169]:
 from catboost import CatBoostClassifier
import catboost
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_log_error

scoring = make_scorer(roc_auc_score)#{'auc': make_scorer(roc_auc_score)}#,
           #'accuracy': make_scorer(accuracy_score)}

# A sample parameter

"""parameters = {
    "loss":["deviance", 'exponential'],
    "learning_rate": [0.01, 0.025],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    #"criterion": ["friedman_mse",  "mae"],
    #"subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[100, 200]
    }"""

parameters = {
    "loss":["deviance", 'exponential'],
    "learning_rate": [0.01, 0.025],
    "min_samples_split": np.linspace(0.1, 0.5, 3),
    "min_samples_leaf": np.linspace(0.1, 0.5, 3),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["auc"],#,  "mae"],
    #"subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[100]#, 200]
    }    
#passing the scoring function in the GridSearchCV
clf = GridSearchCV(CatBoostClassifier(), parameters,scoring=scoring,refit=True, cv=2, n_jobs=-1)
if is_need_pca == True: 

  #model.fit(X_train_pca_part, y_train) 
  clf.fit(train_pool,eval_set=val_pool)
  best_estimator = clf.best_estimator_
  print(best_estimator)
  model=CatBoostClassifier(**best_estimator)
  model.fit(X_train_pca_part, y_train)
  y_pred = model.predict(val_pool) 

else:  
  clf.fit(X_train[column_dict['numeric_column']+column_dict['float_column']],y_train)
  best_estimator = clf.best_estimator_
  print(best_estimator)
  model=CatBoostClassifier(**best_estimator)
  model.fit(X_train[column_dict['numeric_column']+column_dict['float_column']],y_train)
  
  y_pred = model.predict(X_val[column_dict['numeric_column']+column_dict['float_column']]) 

print(f"{roc_auc_score(y_pred, y_val)}")


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
432 fits failed out of a total of 432.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
432 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/catboost/core.py", line 4770, in fit
    silent, early_stopping_rounds, save_snapshot, snapshot_file, snapshot_interval, init_model, callbacks, log_cout, log_cerr)
  File "/usr/local/lib/python3.7/dist-packages/catboost/core.py", line 2081, in _fit
    callbacks=callb

NotFittedError: ignored

In [ ]:
from catboost import CatBoostClassifier
import catboost
from sklearn.metrics import roc_auc_score

#model = CatBoostClassifier()
model = CatBoostClassifier(**CatBoostClassifier_param)

if is_need_pca == True: 
  train_pool = catboost.Pool(
    X_train_pca_part,
    label=y_train,
    cat_features=column_dict['categorical_column']#column_dict['numeric_column']+
  )

  val_pool = catboost.Pool(
    X_val_pca_part,
    label=y_val,
    cat_features=column_dict['categorical_column']#column_dict['float_column']+
  )

  #model.fit(X_train_pca_part, y_train) 
  model.fit(train_pool,eval_set=val_pool)
  y_pred = model.predict(val_pool) 

else:
  #CatBoostClassifier_param['eval_set']=X_val
  train_pool = catboost.Pool(
    X_train,
    label=y_train,
    cat_features=column_dict['categorical_column']#column_dict['numeric_column']+
  )
  val_pool = catboost.Pool(
    X_val,
    label=y_val,
    cat_features=column_dict['categorical_column']#column_dict['float_column']+
  )
  #model = CatBoostClassifier(**CatBoostClassifier_param)
  model.fit(#X_train[column_dict['float_column']+column_dict['numeric_column']], 
            #y_train,
            train_pool,
            eval_set=val_pool) 
  y_pred = model.predict(val_pool)#[column_dict['float_column']+column_dict['numeric_column']]) 

print(f"{roc_auc_score(y_pred, y_val)}")
 

Learning rate set to 0.140329
0:	test: 0.7195632	best: 0.7195632 (0)	total: 6.05s	remaining: 1h 40m 49s
1:	test: 0.7344788	best: 0.7344788 (1)	total: 9.46s	remaining: 1h 18m 39s
2:	test: 0.7680755	best: 0.7680755 (2)	total: 15.1s	remaining: 1h 23m 49s
3:	test: 0.7803427	best: 0.7803427 (3)	total: 22.7s	remaining: 1h 34m 4s
4:	test: 0.7842833	best: 0.7842833 (4)	total: 29.2s	remaining: 1h 36m 44s
5:	test: 0.7875471	best: 0.7875471 (5)	total: 34.6s	remaining: 1h 35m 34s
6:	test: 0.7885049	best: 0.7885049 (6)	total: 40s	remaining: 1h 34m 37s
7:	test: 0.7906844	best: 0.7906844 (7)	total: 43.8s	remaining: 1h 30m 34s
8:	test: 0.8060002	best: 0.8060002 (8)	total: 46.5s	remaining: 1h 25m 18s
9:	test: 0.8119124	best: 0.8119124 (9)	total: 49.2s	remaining: 1h 21m 14s
10:	test: 0.8153199	best: 0.8153199 (10)	total: 52.3s	remaining: 1h 18m 20s
11:	test: 0.8167854	best: 0.8167854 (11)	total: 55.6s	remaining: 1h 16m 14s
12:	test: 0.8170105	best: 0.8170105 (12)	total: 58.5s	remaining: 1h 14m 2s
13:	te

In [ ]:
model.save_model('gdrive/My Drive/data/gp2/model.cbm',
           format="cbm",
           #export_parameters=None,
           pool=train_pool)

NameError: ignored

In [ ]:
if is_need_pca == True: 
  del X_train_pca_part, X_val_pca_part, train_pool, val_pool
  gc.collect
else:
  del X_train, X_val,train_pool,val_pool
  gc.collect

In [ ]:
print(f"{roc_auc_score(y_pred, y_val)}")

0.9274419432587444


Тестовые данные

In [ ]:
model.load_model('gdrive/My Drive/data/gp2/model.cbm',
           format="cbm")

In [ ]:
"""column_dict={'bad_categorical_column': ['id_23', 'id_27'],
 'bad_float_column': ['id_07',
  'id_08',
  'id_21',
  'id_22',
  'id_24',
  'id_25',
  'id_26'],
 'bad_numeric_column': [],
 'categorical_column': ['ProductCD',
  'card4',
  'card6',
  'P_emaildomain',
  'R_emaildomain',
  'M1',
  'M2',
  'M3',
  'M4',
  'M5',
  'M6',
  'M7',
  'M8',
  'M9',
  'id_12',
  'id_15',
  'id_16',
  'id_28',
  'id_29',
  'id_30',
  'id_31',
  'id_33',
  'id_34',
  'id_35',
  'id_36',
  'id_37',
  'id_38'],
 'float_column': ['TransactionAmt',
  'card2',
  'card3',
  'card5',
  'addr1',
  'addr2',
  'dist1',
  'dist2',
  'C1',
  'C2',
  'C3',
  'C4',
  'C5',
  'C6',
  'C7',
  'C8',
  'C9',
  'C10',
  'C11',
  'C12',
  'C13',
  'C14',
  'D1',
  'D2',
  'D3',
  'D4',
  'D5',
  'D6',
  'D7',
  'D8',
  'D9',
  'D10',
  'D11',
  'D12',
  'D13',
  'D14',
  'D15',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6',
  'V7',
  'V8',
  'V9',
  'V10',
  'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V19',
  'V20',
  'V21',
  'V22',
  'V23',
  'V24',
  'V25',
  'V26',
  'V27',
  'V28',
  'V29',
  'V30',
  'V31',
  'V32',
  'V33',
  'V34',
  'V35',
  'V36',
  'V37',
  'V38',
  'V39',
  'V40',
  'V41',
  'V42',
  'V43',
  'V44',
  'V45',
  'V46',
  'V47',
  'V48',
  'V49',
  'V50',
  'V51',
  'V52',
  'V53',
  'V54',
  'V55',
  'V56',
  'V57',
  'V58',
  'V59',
  'V60',
  'V61',
  'V62',
  'V63',
  'V64',
  'V65',
  'V66',
  'V67',
  'V68',
  'V69',
  'V70',
  'V71',
  'V72',
  'V73',
  'V74',
  'V75',
  'V76',
  'V77',
  'V78',
  'V79',
  'V80',
  'V81',
  'V82',
  'V83',
  'V84',
  'V85',
  'V86',
  'V87',
  'V88',
  'V89',
  'V90',
  'V91',
  'V92',
  'V93',
  'V94',
  'V95',
  'V96',
  'V97',
  'V98',
  'V99',
  'V100',
  'V101',
  'V102',
  'V103',
  'V104',
  'V105',
  'V106',
  'V107',
  'V108',
  'V109',
  'V110',
  'V111',
  'V112',
  'V113',
  'V114',
  'V115',
  'V116',
  'V117',
  'V118',
  'V119',
  'V120',
  'V121',
  'V122',
  'V123',
  'V124',
  'V125',
  'V126',
  'V127',
  'V128',
  'V129',
  'V130',
  'V131',
  'V132',
  'V133',
  'V134',
  'V135',
  'V136',
  'V137',
  'V138',
  'V139',
  'V140',
  'V141',
  'V142',
  'V143',
  'V144',
  'V145',
  'V146',
  'V147',
  'V148',
  'V149',
  'V150',
  'V151',
  'V152',
  'V153',
  'V154',
  'V155',
  'V156',
  'V157',
  'V158',
  'V159',
  'V160',
  'V161',
  'V162',
  'V163',
  'V164',
  'V165',
  'V166',
  'V167',
  'V168',
  'V169',
  'V170',
  'V171',
  'V172',
  'V173',
  'V174',
  'V175',
  'V176',
  'V177',
  'V178',
  'V179',
  'V180',
  'V181',
  'V182',
  'V183',
  'V184',
  'V185',
  'V186',
  'V187',
  'V188',
  'V189',
  'V190',
  'V191',
  'V192',
  'V193',
  'V194',
  'V195',
  'V196',
  'V197',
  'V198',
  'V199',
  'V200',
  'V201',
  'V202',
  'V203',
  'V204',
  'V205',
  'V206',
  'V207',
  'V208',
  'V209',
  'V210',
  'V211',
  'V212',
  'V213',
  'V214',
  'V215',
  'V216',
  'V217',
  'V218',
  'V219',
  'V220',
  'V221',
  'V222',
  'V223',
  'V224',
  'V225',
  'V226',
  'V227',
  'V228',
  'V229',
  'V230',
  'V231',
  'V232',
  'V233',
  'V234',
  'V235',
  'V236',
  'V237',
  'V238',
  'V239',
  'V240',
  'V241',
  'V242',
  'V243',
  'V244',
  'V245',
  'V246',
  'V247',
  'V248',
  'V249',
  'V250',
  'V251',
  'V252',
  'V253',
  'V254',
  'V255',
  'V256',
  'V257',
  'V258',
  'V259',
  'V260',
  'V261',
  'V262',
  'V263',
  'V264',
  'V265',
  'V266',
  'V267',
  'V268',
  'V269',
  'V270',
  'V271',
  'V272',
  'V273',
  'V274',
  'V275',
  'V276',
  'V277',
  'V278',
  'V279',
  'V280',
  'V281',
  'V282',
  'V283',
  'V284',
  'V285',
  'V286',
  'V287',
  'V288',
  'V289',
  'V290',
  'V291',
  'V292',
  'V293',
  'V294',
  'V295',
  'V296',
  'V297',
  'V298',
  'V299',
  'V300',
  'V301',
  'V302',
  'V303',
  'V304',
  'V305',
  'V306',
  'V307',
  'V308',
  'V309',
  'V310',
  'V311',
  'V312',
  'V313',
  'V314',
  'V315',
  'V316',
  'V317',
  'V318',
  'V319',
  'V320',
  'V321',
  'V322',
  'V323',
  'V324',
  'V325',
  'V326',
  'V327',
  'V328',
  'V329',
  'V330',
  'V331',
  'V332',
  'V333',
  'V334',
  'V335',
  'V336',
  'V337',
  'V338',
  'V339',
  'id_01',
  'id_02',
  'id_03',
  'id_04',
  'id_05',
  'id_06',
  'id_09',
  'id_10',
  'id_11',
  'id_13',
  'id_14',
  'id_17',
  'id_18',
  'id_19',
  'id_20',
  'id_32'],
 'numeric_column': ['card1']}"""

{'bad_categorical_column': ['id_23', 'id_27'],
 'bad_float_column': ['id_07',
  'id_08',
  'id_21',
  'id_22',
  'id_24',
  'id_25',
  'id_26'],
 'bad_numeric_column': [],
 'categorical_column': ['ProductCD',
  'card4',
  'card6',
  'P_emaildomain',
  'R_emaildomain',
  'M1',
  'M2',
  'M3',
  'M4',
  'M5',
  'M6',
  'M7',
  'M8',
  'M9',
  'id_12',
  'id_15',
  'id_16',
  'id_28',
  'id_29',
  'id_30',
  'id_31',
  'id_33',
  'id_34',
  'id_35',
  'id_36',
  'id_37',
  'id_38'],
 'float_column': ['TransactionAmt',
  'card2',
  'card3',
  'card5',
  'addr1',
  'addr2',
  'dist1',
  'dist2',
  'C1',
  'C2',
  'C3',
  'C4',
  'C5',
  'C6',
  'C7',
  'C8',
  'C9',
  'C10',
  'C11',
  'C12',
  'C13',
  'C14',
  'D1',
  'D2',
  'D3',
  'D4',
  'D5',
  'D6',
  'D7',
  'D8',
  'D9',
  'D10',
  'D11',
  'D12',
  'D13',
  'D14',
  'D15',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6',
  'V7',
  'V8',
  'V9',
  'V10',
  'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V19

In [ ]:
df_test, df_y1 = get_data('gdrive/My Drive/data/gp2/test_transaction.csv', 
                'gdrive/My Drive/data/gp2/test_identity.csv',
                True,delete_column_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


Исправление названий признаков

In [ ]:
dict_col = {}
for column in df_test.columns:
  if column.startswith('id-'):
    dict_col[column] = column.replace("-","_")
df_test.rename(columns = dict_col,inplace=True)   

Отказ от всех лишних признаков 

In [ ]:
column_dict

{'bad_categorical_column': ['id_23', 'id_27'],
 'bad_float_column': ['id_07',
  'id_08',
  'id_21',
  'id_22',
  'id_24',
  'id_25',
  'id_26'],
 'bad_numeric_column': [],
 'categorical_column': ['ProductCD',
  'card4',
  'card6',
  'P_emaildomain',
  'R_emaildomain',
  'M1',
  'M2',
  'M3',
  'M4',
  'M5',
  'M6',
  'M7',
  'M8',
  'M9',
  'id_12',
  'id_15',
  'id_16',
  'id_28',
  'id_29',
  'id_30',
  'id_31',
  'id_33',
  'id_34',
  'id_35',
  'id_36',
  'id_37',
  'id_38'],
 'float_column': ['TransactionAmt',
  'card2',
  'card3',
  'card5',
  'addr1',
  'addr2',
  'dist1',
  'dist2',
  'C1',
  'C2',
  'C3',
  'C4',
  'C5',
  'C6',
  'C7',
  'C8',
  'C9',
  'C10',
  'C11',
  'C12',
  'C13',
  'C14',
  'D1',
  'D2',
  'D3',
  'D4',
  'D5',
  'D6',
  'D7',
  'D8',
  'D9',
  'D10',
  'D11',
  'D12',
  'D13',
  'D14',
  'D15',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6',
  'V7',
  'V8',
  'V9',
  'V10',
  'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V19

In [ ]:
df_test = df_test[column_dict['float_column']+column_dict['numeric_column']+\
                  column_dict['categorical_column']]
"""+\
                  column_dict['bad_float_column']+\
                  column_dict['bad_numeric_column']+\
                  column_dict['bad_categorical_column']]"""

Заполнение пустых ячеек

In [ ]:
"""for column in column_dict['bad_categorical_column']:
  df_test[column].fillna(bad_fill_category_value, inplace = True)
for column in column_dict['bad_float_column']:
  df_test[column].fillna(bad_fill_float_value, inplace = True)
for column in column_dict['bad_numeric_column']:
  df_test[column].fillna(bad_fill_numeric_value, inplace = True)


for column in column_dict['categorical_column']:
  if not(column in column_dict['bad_categorical_column']):
    df_test[column].fillna(bad_fill_category_value, inplace = True)
for column in column_dict['float_column']:
  if not(column in column_dict['bad_float_column']):
    df_test[column].fillna(df_test[column].mean(), inplace = True)
for column in column_dict['numeric_column']:
  if not(column in column_dict['bad_numeric_column']):
    df_test[column].fillna(df_test[column].mean(), inplace = True)"""


In [ ]:
df_test,column_dict = fill_data(df_test,column_dict,is_drop_nan_columns, True)

Применение PCA

In [ ]:
if is_need_pca == True:
  X_test_pca_part1 = pca.transform(df_test[column_dict['float_column']+column_dict['numeric_column']])

  X_test_pca_part = pd.DataFrame(X_test_pca_part1,index=df_test.index)  
  X_test_pca_part = X_test_pca_part.join(df_test[column_dict['categorical_column']])
 

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: ignored

Применение модели

In [ ]:
if is_need_pca == True:
  test_pool = catboost.Pool(
    X_test_pca_part,
    cat_features=column_dict['categorical_column']#column_dict['float_column']+
  )

  Y_test_pred = model.predict(test_pool)
else:
  test_pool = catboost.Pool(
    df_test,
    cat_features=column_dict['categorical_column']#column_dict['float_column']+
  )
  Y_test_pred = model.predict(test_pool)#df_test[column_dict['float_column']+column_dict['numeric_column']])


In [ ]:
del df_test,test_pool

Формирование датасета результата

In [ ]:
submission = pd.DataFrame(columns=['TransactionID'], data=df_y1 )
submission[target] = Y_test_pred
submission.head()
submission.isFraud.value_counts()

Сохранение файла

In [ ]:
submission.to_csv("submission.csv", index = False)